# Interactive Cropping Tool Demo

In [60]:
# import interactive_crop
import os
image_list = ['image_samples/'+i for i in os.listdir('image_samples')]
main(image_list)

GridBox(children=(Dropdown(description='Image Name:', layout=Layout(grid_area='im_selector'), options=('image_…

In [59]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, Layout, GridBox, fixed
from matplotlib import pyplot as plt
from matplotlib import patches
from IPython.display import display, Markdown
import time
from PIL import Image, ImageDraw
from sys import exit as sysexit
import numpy as np

# Change default output size of matplotlib figures
plt.rcParams['figure.figsize'] = (13.0, 11.0)
plt.rcParams['figure.dpi'] = 100 # default for me was 75

# Need these to retain the global scope
def main(image_list, shape = 'Rectangle', continuous_update=True,
    callback = lambda x,y: print('{}: {}'.format(x,y))):
    """
    This function takes a list of images and allows the user interactively crop these images through a vertical range slider and a horizontal range slider. Once the crop size is accepted, the callback kwarg will be called and provided the name of the cropped image (from image_list) and the shape of the cropped_image.

    Arguments: 
    image_list - A list of image names
    shape - The desired type of crop shape
    callback - The function to call after the crop size is accepted

    Returns: 
    coord_dict - A dict mapping image names to cropped coordinates
    """
    coord_dict = {}
    def show_image(image_name):
        """
        Shows the image at the desired size
        """
        global im
        global im_draw
        # Read image from file
        im = Image.open(image_name)
        
        # Create copy of image to draw on
        im_draw =im.copy()
        
        # Adjust the slider limits based on the pixel size of the image
        x_size_selector.min, x_size_selector.max = 0, im.size[0]
        x_mod.min, x_mod.max = 0, im.size[0]
        y_size_selector.min, y_size_selector.max = 0, im.size[1]
        y_mod.min, y_mod.max = 0, im.size[1]
        y_size_selector.value = (int(im.size[1]/3), int(2*im.size[1]/3))
        x_size_selector.value = (int(im.size[0]/3), int(2*im.size[0]/3))
        
        # Get the axs and imshow on the existing axs
        def add_crop_patch(x_size, y_size):
            global im
            global im_draw
            # Parse the inputs from the sliders into rectangle form 
            colstart= x_size[0]
            width = x_size[1]-x_size[0]
            rowstart = im.size[1]-y_size[1]
            height = y_size[1]-y_size[0]
            
            # Draw the rectangle on im_draw
            draw = ImageDraw.Draw(im_draw)
            line_width = int(np.min(im.size)/100)
            draw.rectangle((colstart, rowstart, colstart+width, rowstart+height), width=line_width)
#             display(Markdown('### <div style="text-align:center">'+image_name+'</div>'))
            display(im_draw)
            im_draw = im.copy()

        current_size = interact(add_crop_patch, x_size=x_mod, y_size = y_mod)
        
        # Click button to save params
        save_crop_sizes=interact.options(manual=True, manual_name="Save Crop Sizes")
        @save_crop_sizes
        def on_button_click(image_name=fixed(image_name)):
            """
            This will handle the onbutton click event of the cropper.
            """
            # Compute the size based on the value of x_mod and y_mod
            x_size, y_size = x_mod.value, y_mod.value
            colstart= x_size[0]
            width = x_size[1]-x_size[0]
            rowstart = im.shape[0]-y_size[1]
            height = y_size[1]-y_size[0]
            size = (colstart, rowstart, width, height)
            
            # Call the callback function
            callback(image_name, size)
            coord_dict[image_name] = size
            im_index = image_list.index(image_name)
            if im_index != len(image_list)-1:
                image_selector.value = image_list[im_index+1]
            else:
                print('Made it through all the tests; Stopping execution now.')
                sysexit()
            time.sleep(5) # Change this to change how long you have to look at the cropped results
            return
        return {'image':im, 'size':current_size}
    # Define test_name selector
    image_selector = widgets.Dropdown(
    options=image_list,
    value=image_list[0],
    description='Image Name:',
    disabled=False,
    continuous_update=continuous_update,
    layout=Layout(grid_area='im_selector')
    )

    # Define selectors for int range sliders; We'll define value and max, min 
    # once we know image size
    x_size_selector = widgets.IntRangeSlider(
        step=1,
        description='Width: ',
        continuous_update=continuous_update,
        orientation='horizontal',
        layout=Layout(width='auto', grid_area='width')
    )

    y_size_selector = widgets.IntRangeSlider(
        step=1,
        description='Height: ',
        continuous_update=continuous_update,
        orientation='vertical',
#         readout=True,
        layout=Layout(width='auto',height = 'auto', grid_area='height')
    )
    # Define mod sliders for cropping function; We'll define value and max, min 
    # once we know image size
    x_mod = widgets.IntRangeSlider(
        layout=Layout(width='0%',height='0%', visibility='hidden')
    )
    y_mod = widgets.IntRangeSlider(
        layout=Layout(width='0%',height='0%', visibility='hidden')
    )
    im_mod = widgets.Select(
    options=image_list,
    value=image_list[0],
    layout=Layout(width='0%',height='0%', visibility='hidden')
    )

    # Link the x,y,im selectors to mod widgets for the interactive function
    widgetLinkx = widgets.jslink((x_size_selector, 'value'), (x_mod, 'value'))
    widgetLinky = widgets.jslink((y_size_selector, 'value'), (y_mod, 'value'))
    widgetLinktn = widgets.jslink((image_selector, 'index'), (im_mod, 'index'))

    cropper = interactive(show_image, image_name =im_mod, layout=Layout(width='auto', grid_area='main'));
    return GridBox(children=[image_selector,x_size_selector,y_size_selector, cropper ],
        layout=Layout(
            width='90%',
            grid_template_rows='10% 90%',
            grid_template_columns='30% 70%',
            grid_template_areas='''
            "im_selector width"
            "height main "
            '''))

In [12]:
(0,0)+(100,100)

(0, 0, 100, 100)

In [32]:
from ipywidgets import Image

file = open("image_samples/elephant.jpg", "rb")
image = file.read()
# help(image)

im_widg = Image(
value=image,
format='png',
width=500,
height=1200,
)
# help(im_widg)

In [33]:
%%capture
%matplotlib inline
from ipywidgets import interact, widgets
from IPython.display import display
import matplotlib.pyplot as plt
import warnings

from skimage.morphology import disk
from skimage.filters import rank
from skimage.color import rgb2gray
import skimage.data

def f(Median_Size):
    selem = disk(int(Median_Size))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        img_median = rank.median(img_gray, selem=selem) 

    ax_neu.imshow(img_median, cmap="gray")
    fig.canvas.draw()
    display(fig)

image = skimage.data.camera() #plt.imread("Test.png")       
img_gray = rgb2gray(image)

fig = plt.figure(figsize=(6, 4))
ax_orig = fig.add_subplot(121) 
ax_neu = fig.add_subplot(122) 

ax_orig.imshow(img_gray, cmap="gray")
ax_neu.imshow(img_gray, cmap="gray")

In [34]:
interact(f, Median_Size=widgets.IntSlider(min=1,max=21,step=2,value=1));

interactive(children=(IntSlider(value=1, description='Median_Size', max=21, min=1, step=2), Output()), _dom_cl…

In [2]:
# import interactive_crop
import os
image_list = ['image_samples/'+i for i in os.listdir('image_samples')]
main(image_list)

0

GridBox(children=(Dropdown(description='Test Name:', layout=Layout(grid_area='test_name'), options=('image_sam…